In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# !pip install pyradiomics

     |████████████████████████████████| 163kB 2.7MB/s 
     |████████████████████████████████| 4.4MB 44.8MB/s 
     |████████████████████████████████| 42.5MB 104kB/s 
     |████████████████████████████████| 40kB 6.2MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: PyWavelets 1.1.1
    Uninstalling PyWavelets-1.1.1:
      Successfully uninstalled PyWavelets-1.1.1


In [0]:
# from radiomics import featureextractor
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import SimpleITK as sitk
import six
import sys, os
import numpy as np
import cv2
import pandas as pd 

In [0]:
n_images = 2000

In [0]:
df=pd.read_csv('/content/gdrive/My Drive/ Data_Entry_2017.csv')

df2 = df[['Image Index','Finding Labels']]
df2 = df2[0:n_images][:]

In [0]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Image Index     2000 non-null   object
 1   Finding Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [0]:
mask = np.full((1024,1024), 1)
mask[0][0:] = 0
mask[1023][0:] = 0
mask = np.transpose(mask)
mask[0][0:] = 0
mask[1023][0:] = 0

In [0]:
m = cv2.imwrite('mask.png', mask)

In [0]:
imgfolder = '/content/gdrive/My Drive/images/' 
m = sitk.ReadImage('mask.png')

In [0]:
kurtosis = []
mean = []
skewness = []
variance = []

In [0]:
for idx in range(n_images):
  print("idx",idx)
  img = df2['Image Index'][idx]
  path = imgfolder + img

  i = sitk.ReadImage(path)
  map_key = "0"
  try:
    selector = sitk.VectorIndexSelectionCastImageFilter()
    selector.SetIndex(0)
    i = selector.Execute(i)
    map_key = "1"
  except RuntimeError:
    pass

  firstOrderFeatures = firstorder.RadiomicsFirstOrder(i, m)
  firstOrderFeatures.enableAllFeatures()  # On the feature class level, all features are disabled by default.
  result = firstOrderFeatures.execute()

  KEY_MAP = {
      "kurtosis": {
          "0": "Kurtosis",
          "1": "original_firstorder_Kurtosis"
      },
      "mean": {
          "0": "Mean",
          "1": "original_firstorder_Mean"
      },
      "skewness": {
          "0": "Skewness",
          "1": "original_firstorder_Skewness"
      },
      "variance": {
          "0": "Variance",
          "1": "original_firstorder_Variance"
      }
  }

  kurtosis.append(result.get(KEY_MAP['kurtosis'][map_key]))
  mean.append(result.get(KEY_MAP['mean'][map_key]))
  skewness.append(result.get(KEY_MAP['skewness'][map_key]))
  variance.append(result.get(KEY_MAP['variance'][map_key]))
  # for (key,val) in six.iteritems(result):
  #   if key == KEY_MAP['kurtosis'][map_key]:
  #     kurtosis.append(val)
  #   elif key == KEY_MAP['mean'][map_key]:
  #     mean.append(val)
  #   elif key == KEY_MAP['skewness'][map_key]:
  #     skewness.append(val)
  #   elif key == KEY_MAP['variance'][map_key]:
  #     variance.append(val)

idx 0
idx 1
idx 2
idx 3
idx 4
idx 5
idx 6
idx 7
idx 8
idx 9
idx 10
idx 11
idx 12
idx 13
idx 14
idx 15
idx 16
idx 17
idx 18
idx 19
idx 20
idx 21
idx 22
idx 23
idx 24
idx 25
idx 26
idx 27
idx 28
idx 29
idx 30
idx 31
idx 32
idx 33
idx 34
idx 35
idx 36
idx 37
idx 38
idx 39
idx 40
idx 41
idx 42
idx 43
idx 44
idx 45
idx 46
idx 47
idx 48
idx 49
idx 50
idx 51
idx 52
idx 53
idx 54
idx 55
idx 56
idx 57
idx 58
idx 59
idx 60
idx 61
idx 62
idx 63
idx 64
idx 65
idx 66
idx 67
idx 68
idx 69
idx 70
idx 71
idx 72
idx 73
idx 74
idx 75
idx 76
idx 77
idx 78
idx 79
idx 80
idx 81
idx 82
idx 83
idx 84
idx 85
idx 86
idx 87
idx 88
idx 89
idx 90
idx 91
idx 92
idx 93
idx 94
idx 95
idx 96
idx 97
idx 98
idx 99
idx 100
idx 101
idx 102
idx 103
idx 104
idx 105
idx 106
idx 107
idx 108
idx 109
idx 110
idx 111
idx 112
idx 113
idx 114
idx 115
idx 116
idx 117
idx 118
idx 119
idx 120
idx 121
idx 122
idx 123
idx 124
idx 125
idx 126
idx 127
idx 128
idx 129
idx 130
idx 131
idx 132
idx 133
idx 134
idx 135
idx 136
idx 137
idx 13

In [0]:
len(kurtosis), len(mean), len(skewness), len(variance)

(2000, 2000, 2000, 2000)

In [0]:
df2['Kurtosis'] = kurtosis
df2['Mean'] = mean
df2['Skewness'] = skewness
df2['Variance'] = variance

In [0]:
df2[df2['Kurtosis'] == None]

,Image Index,Finding Labels,Kurtosis,Mean,Skewness,Variance


In [0]:
df2.to_csv('/content/gdrive/My Drive/DataFiles/STAT_basic.csv')

# DIVIDED IMAGES

# CROPPPED IMAGES

In [0]:
kurtosis_c = []
mean_c = []
skewness_c = []
variance_c = []

kurtosis_dl = []
mean_dl = []
skewness_dl = []
variance_dl = []

kurtosis_dr = []
mean_dr = []
skewness_dr = []
variance_dr = []

kurtosis_cdl = []
mean_cdl = []
skewness_cdl = []
variance_cdl = []

kurtosis_cdr = []
mean_cdr = []
skewness_cdr = []
variance_cdr = []

In [0]:
crop_m = cv2.imread('mask.png')
divide_m = cv2.cvtColor(crop_m, cv2.COLOR_BGR2GRAY)
crop_m = cv2.cvtColor(crop_m, cv2.COLOR_BGR2GRAY)

cropped_m = crop_m[100:-100, 100:-100]
cv2.imwrite('cropped_mask.png',cropped_m)
cm = sitk.ReadImage('cropped_mask.png')

divided_ml = divide_m[0:1024 , 0:512]
cv2.imwrite('divided_mask_left.png',divided_ml)
lm = sitk.ReadImage('divided_mask_left.png')

divided_mr = divide_m[0:1024 , 512:1024]
cv2.imwrite('divided_mask_right.png',divided_mr)
rm = sitk.ReadImage('divided_mask_right.png')

cropDivided_ml = cropped_m[0:1024 , 0:512]
cv2.imwrite('cropDivided_mask_left.png',cropDivided_ml)
clm = sitk.ReadImage('cropDivided_mask_left.png')

cropDivided_mr = cropped_m[0:1024 , 512:1024]
cv2.imwrite('cropDivided_mask_right.png',cropDivided_mr)
crm = sitk.ReadImage('cropDivided_mask_right.png')


In [13]:
for idx in range(n_images):
  print("idx",idx)

  img = df2['Image Index'][idx]
  path = imgfolder + img

  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_copy = img.copy()

  # crop image
  cropped_img = img[100:-100, 100:-100]
  cropped_img_copy = cropped_img.copy()

  # divide image
  divided_img_left = img_copy[0:1024 , 0:512]
  divided_img_right = img_copy[0:1024 , 512:1024]

  # cropped divided image
  crop_divided_img_left = cropped_img_copy[0:1024 , 0:512]
  crop_divided_img_right = cropped_img_copy[0:1024 , 512:1024]

  # save images
  cv2.imwrite('cropped_img.png',cropped_img)
  cv2.imwrite('divided_img_left.png',divided_img_left)
  cv2.imwrite('divided_img_right.png',divided_img_right)
  cv2.imwrite('crop_divided_img_left.png',crop_divided_img_left)
  cv2.imwrite('crop_divided_img_right.png',crop_divided_img_right)

  # read images
  c = sitk.ReadImage('cropped_img.png')
  dl = sitk.ReadImage('divided_img_left.png')
  dr = sitk.ReadImage('divided_img_right.png')
  cdl = sitk.ReadImage('crop_divided_img_left.png')
  cdr = sitk.ReadImage('crop_divided_img_right.png')


  map_key = "0"
  try:
    selector = sitk.VectorIndexSelectionCastImageFilter()
    selector.SetIndex(0)

    c = selector.Execute(c)
    dl = selector.Execute(dl)
    dr = selector.Execute(dr)
    cdl = selector.Execute(cdl)
    cdr = selector.Execute(cdr)

    map_key = "1"
  except RuntimeError:
    pass


  firstOrderFeatures_c = firstorder.RadiomicsFirstOrder(c, cm)
  firstOrderFeatures_dl = firstorder.RadiomicsFirstOrder(dl, lm)
  firstOrderFeatures_dr = firstorder.RadiomicsFirstOrder(dr, rm)
  firstOrderFeatures_cdl = firstorder.RadiomicsFirstOrder(cdl, clm)
  firstOrderFeatures_cdr = firstorder.RadiomicsFirstOrder(cdr, crm)

  firstOrderFeatures_c.enableAllFeatures()  # On the feature class level, all features are disabled by default.
  firstOrderFeatures_dl.enableAllFeatures()  # On the feature class level, all features are disabled by default.
  firstOrderFeatures_dr.enableAllFeatures()  # On the feature class level, all features are disabled by default.
  firstOrderFeatures_cdl.enableAllFeatures()  # On the feature class level, all features are disabled by default.
  firstOrderFeatures_cdr.enableAllFeatures()  # On the feature class level, all features are disabled by default.

  result_c = firstOrderFeatures_c.execute()
  result_dl = firstOrderFeatures_dl.execute()
  result_dr = firstOrderFeatures_dr.execute()
  result_cdl = firstOrderFeatures_cdl.execute()
  result_cdr = firstOrderFeatures_cdr.execute()

  KEY_MAP = {
      "kurtosis": {
          "0": "Kurtosis",
          "1": "original_firstorder_Kurtosis"
      },
      "mean": {
          "0": "Mean",
          "1": "original_firstorder_Mean"
      },
      "skewness": {
          "0": "Skewness",
          "1": "original_firstorder_Skewness"
      },
      "variance": {
          "0": "Variance",
          "1": "original_firstorder_Variance"
      }
  }

  kurtosis_c.append(result_c.get(KEY_MAP['kurtosis'][map_key]))
  mean_c.append(result_c.get(KEY_MAP['mean'][map_key]))
  skewness_c.append(result_c.get(KEY_MAP['skewness'][map_key]))
  variance_c.append(result_c.get(KEY_MAP['variance'][map_key]))

  kurtosis_dl.append(result_dl.get(KEY_MAP['kurtosis'][map_key]))
  mean_dl.append(result_dl.get(KEY_MAP['mean'][map_key]))
  skewness_dl.append(result_dl.get(KEY_MAP['skewness'][map_key]))
  variance_dl.append(result_dl.get(KEY_MAP['variance'][map_key]))
  
  kurtosis_dr.append(result_dr.get(KEY_MAP['kurtosis'][map_key]))
  mean_dr.append(result_dr.get(KEY_MAP['mean'][map_key]))
  skewness_dr.append(result_dr.get(KEY_MAP['skewness'][map_key]))
  variance_dr.append(result_dr.get(KEY_MAP['variance'][map_key]))

  kurtosis_cdl.append(result_cdl.get(KEY_MAP['kurtosis'][map_key]))
  mean_cdl.append(result_cdl.get(KEY_MAP['mean'][map_key]))
  skewness_cdl.append(result_cdl.get(KEY_MAP['skewness'][map_key]))
  variance_cdl.append(result_cdl.get(KEY_MAP['variance'][map_key]))

  kurtosis_cdr.append(result_cdr.get(KEY_MAP['kurtosis'][map_key]))
  mean_cdr.append(result_cdr.get(KEY_MAP['mean'][map_key]))
  skewness_cdr.append(result_cdr.get(KEY_MAP['skewness'][map_key]))
  variance_cdr.append(result_cdr.get(KEY_MAP['variance'][map_key]))

idx 0
idx 1
idx 2
idx 3
idx 4
idx 5
idx 6
idx 7
idx 8
idx 9
idx 10
idx 11
idx 12
idx 13
idx 14
idx 15
idx 16
idx 17
idx 18
idx 19
idx 20
idx 21
idx 22
idx 23
idx 24
idx 25
idx 26
idx 27
idx 28
idx 29
idx 30
idx 31
idx 32
idx 33
idx 34
idx 35
idx 36
idx 37
idx 38
idx 39
idx 40
idx 41
idx 42
idx 43
idx 44
idx 45
idx 46
idx 47
idx 48
idx 49
idx 50
idx 51
idx 52
idx 53
idx 54
idx 55
idx 56
idx 57
idx 58
idx 59
idx 60
idx 61
idx 62
idx 63
idx 64
idx 65
idx 66
idx 67
idx 68
idx 69
idx 70
idx 71
idx 72
idx 73
idx 74
idx 75
idx 76
idx 77
idx 78
idx 79
idx 80
idx 81
idx 82
idx 83
idx 84
idx 85
idx 86
idx 87
idx 88
idx 89
idx 90
idx 91
idx 92
idx 93
idx 94
idx 95
idx 96
idx 97
idx 98
idx 99
idx 100
idx 101
idx 102
idx 103
idx 104
idx 105
idx 106
idx 107
idx 108
idx 109
idx 110
idx 111
idx 112
idx 113
idx 114
idx 115
idx 116
idx 117
idx 118
idx 119
idx 120
idx 121
idx 122
idx 123
idx 124
idx 125
idx 126
idx 127
idx 128
idx 129
idx 130
idx 131
idx 132
idx 133
idx 134
idx 135
idx 136
idx 137
idx 13

In [0]:
df_c = pd.DataFrame()
df_dl = pd.DataFrame()
df_dr = pd.DataFrame()
df_cdl = pd.DataFrame()
df_cdr = pd.DataFrame()

In [0]:
df_c['Kurtosis'] = kurtosis_c
df_c['Mean'] = mean_c
df_c['Skewness'] = skewness_c
df_c['Variance'] = variance_c

df_dl['Kurtosis'] = kurtosis_dl
df_dl['Mean'] = mean_dl
df_dl['Skewness'] = skewness_dl
df_dl['Variance'] = variance_dl

df_dr['Kurtosis'] = kurtosis_dr
df_dr['Mean'] = mean_dr
df_dr['Skewness'] = skewness_dr
df_dr['Variance'] = variance_dr

df_cdl['Kurtosis'] = kurtosis_cdl
df_cdl['Mean'] = mean_cdl
df_cdl['Skewness'] = skewness_cdl
df_cdl['Variance'] = variance_cdl

df_cdr['Kurtosis'] = kurtosis_cdr
df_cdr['Mean'] = mean_cdr
df_cdr['Skewness'] = skewness_cdr
df_cdr['Variance'] = variance_cdr

In [0]:
df_dl['Kurtosis'] = abs(df_dl['Kurtosis'] - df_dr['Kurtosis'])
df_dl['Mean'] = abs(df_dl['Mean'] - df_dr['Mean'])
df_dl['Skewness'] = abs(df_dl['Skewness'] - df_dr['Skewness'])
df_dl['Variance'] = abs(df_dl['Variance'] - df_dr['Variance'])

df_cdl['Kurtosis'] = abs(df_cdl['Kurtosis'] - df_cdr['Kurtosis'])
df_cdl['Mean'] = abs(df_cdl['Mean'] - df_cdr['Mean'])
df_cdl['Skewness'] = abs(df_cdl['Skewness'] - df_cdr['Skewness'])
df_cdl['Variance'] = abs(df_cdl['Variance'] - df_cdr['Variance'])

In [0]:
df_c = pd.concat([df2,df_c],axis = 1)
df_dl = pd.concat([df2,df_dl],axis = 1)
df_cdl = pd.concat([df2,df_cdl],axis = 1)

In [19]:
df_c.head()

,Image Index,Finding Labels,Kurtosis,Mean,Skewness,Variance
0,00000001_000.png,Cardiomegaly,1.7250932190801185,131.8022507422943,-0.1361627318847728,3830.444233375576
1,00000001_001.png,Cardiomegaly|Emphysema,1.945973442653254,126.62177750023565,-0.14193930432168875,3415.2323159421994
2,00000001_002.png,Cardiomegaly|Effusion,2.8429520793285477,179.6506282990857,-0.7809287715246753,2766.902003593599
3,00000002_000.png,No Finding,2.4277064058245355,181.82594966537846,-0.7327606514223735,2105.1478865639665
4,00000003_000.png,Hernia,1.6095097296772405,133.81160747949855,-0.16748703186781808,4497.80972222749


In [20]:
df_dl.head()

,Image Index,Finding Labels,Kurtosis,Mean,Skewness,Variance
0,00000001_000.png,Cardiomegaly,0.0560511,4.75527,0.0244738,187.388
1,00000001_001.png,Cardiomegaly|Emphysema,0.183915,17.0126,0.317436,232.7
2,00000001_002.png,Cardiomegaly|Effusion,1.2461,25.7026,0.515364,1064.28
3,00000002_000.png,No Finding,0.665818,14.1073,0.263696,1027.09
4,00000003_000.png,Hernia,0.0954794,7.96884,0.0706883,189.191


In [21]:
df_cdl.head()

,Image Index,Finding Labels,Kurtosis,Mean,Skewness,Variance
0,00000001_000.png,Cardiomegaly,0.153065,13.0194,0.0599803,614.715
1,00000001_001.png,Cardiomegaly|Emphysema,0.151665,6.53816,0.00134074,448.252
2,00000001_002.png,Cardiomegaly|Effusion,0.314855,6.4495,0.165618,858.866
3,00000002_000.png,No Finding,0.371195,24.553,0.181312,381.922
4,00000003_000.png,Hernia,0.11197,9.20573,0.169361,1091.06


# save in csvs

In [0]:
# cropped
df_c.to_csv('/content/gdrive/My Drive/DataFiles/STAT_crop.csv')

# divided
df_dl.to_csv('/content/gdrive/My Drive/DataFiles/STAT_divide.csv')

# cropped and divided
df_cdl.to_csv('/content/gdrive/My Drive/DataFiles/STAT_cropDivide.csv')

In [0]:
STATbasic = pd.read_csv('/content/gdrive/My Drive/DataFiles/First Order Features/STAT_basic.csv')

basic = pd.read_csv('/content/gdrive/My Drive/DataFiles/Image Features of 2k Images/basicGlcm_2k.csv')
crop = pd.read_csv('/content/gdrive/My Drive/DataFiles/Image Features of 2k Images/cropGlcm_2k.csv')
divide = pd.read_csv('/content/gdrive/My Drive/DataFiles/Image Features of 2k Images/Division_data_2k.csv')
cropDivide = pd.read_csv('/content/gdrive/My Drive/DataFiles/Image Features of 2k Images/cropAndDivideGlcm_2k.csv')

In [0]:
basic = basic[basic.columns[3:]]
crop = crop[crop.columns[3:]]
divide = divide[divide.columns[3:]]
cropDivide = cropDivide[cropDivide.columns[3:]]

In [0]:
STATbasic = STATbasic[STATbasic.columns[1:]]

In [0]:
data_basic = pd.concat([STATbasic, basic], axis=1)
data_crop = pd.concat([df_c, crop], axis=1)
data_divide = pd.concat([df_dl, divide],axis=1)
data_crop_divide = pd.concat([df_cdl, cropDivide],axis=1)

In [0]:
# basic
data_basic.to_csv('/content/gdrive/My Drive/DataFiles/STAT_GLCM_basic.csv')

# cropped
data_crop.to_csv('/content/gdrive/My Drive/DataFiles/STAT_GLCM_crop.csv')

# divided
data_divide.to_csv('/content/gdrive/My Drive/DataFiles/STAT_GLCM_divide.csv')

# cropped and divided
data_crop_divide.to_csv('/content/gdrive/My Drive/DataFiles/STAT_GLCM_cropDivide.csv')